反序列化OpenQASM文件对象，并运行量子线路。

In [5]:
"""Deserialize OpenQASM file object."""
from qutrunk.circuit import QCircuit

circuit = QCircuit.load(file="bell_pair.qasm", format="openqasm")
# run circuit
res = circuit.run(shots=100)
# print result
print(res.get_measure())
print(res.get_counts())

[0, 0]
[{"00": 49}, {"11": 51}]


反序列化QuSL文件对象，并运行量子线路。

In [6]:
"""Deserialize QuSL file object."""
from qutrunk.circuit import QCircuit

circuit = QCircuit.load(file="bell_pair.qusl")
# run circuit
res = circuit.run(shots=100)
# print result
print(res.get_measure())
print(res.get_counts())

[0, 0]
[{"00": 52}, {"11": 48}]


使用qusprout作为后端运行量子线路

In [8]:
from qutrunk.circuit import QCircuit
from qutrunk.backends import BackendQuSprout
from qutrunk.circuit.gates import H, CNOT, Measure

# allocate
qc = QCircuit(backend=BackendQuSprout())
qr = qc.allocate(2)

# apply gate
H * qr[0]
CNOT * (qr[0], qr[1])
Measure * qr[0]
Measure * qr[1]

# run circuit
res = qc.run(shots=100)

# print result
print(res.get_counts())

[{"00": 47}, {"11": 53}]


以OpenQASM格式导出量子线路

In [9]:
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, CNOT, Measure

# allocate
qc = QCircuit()
qr = qc.allocate(2)

# apply gate
H * qr[0]
CNOT * (qr[0], qr[1])
Measure * qr[0]
Measure * qr[1]

# export
qc.dump(file="bell_pair.qasm", format="openqasm")

with open(file="bell_pair.qasm") as f:
    for line in f:
        print(line, end="")

OPENQASM 2.0;
include "qulib1.inc";
qreg q[2];
creg c[2];
h q[0];
cx q[0],q[1];
measure q[0] -> c[0];
measure q[1] -> c[1];


应用振幅放大：先对4个量子比特做均匀叠加，然后选择状态值为7作为标记值，做3次QAA迭代计算，运行后得到的结果是7对应3状态出现的概率超过了96%

In [10]:
from qutrunk.circuit.ops import QAA
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, All

circuit = QCircuit()
qureg = circuit.allocate(4)
All(H) * qureg
QAA(3, 7) * qureg
for i in range(2 ** len(qureg)):
    print(circuit.get_prob_amp(i))

prob of state |7> = 0.4726562499999991
prob of state |7> = 0.9084472656249968
prob of state |7> = 0.9613189697265575
0.002578735351562488
0.0025787353515624874
0.002578735351562488
0.0025787353515624874
0.002578735351562488
0.0025787353515624874
0.002578735351562488
0.9613189697265575
0.002578735351562488
0.0025787353515624874
0.002578735351562488
0.0025787353515624874
0.002578735351562488
0.0025787353515624874
0.002578735351562488
0.0025787353515624853


使用QSP制备量子状态

In [11]:
from qutrunk.circuit.ops import QSP
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, All, Measure

circuit = QCircuit()
qureg = circuit.allocate(2)
QSP("+") * qureg
circuit.print(unroll=False)
print(circuit.get_all_state())

qreg q[2]
creg c[2]
QSP('+') * q
['0.4999999999999999, 0', '0.4999999999999999, 0', '0.4999999999999999, 0', '0.4999999999999999, 0']


申请多个量子寄存器

In [12]:
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, CNOT, Measure

qc = QCircuit()
# allocate 2 qureg: q1, q2
q1, q2 = qc.allocate([1, 1])

H * q1[0]
CNOT * (q1[0], q2[0])
Measure * q1[0]
Measure * q2[0]
res = qc.run(shots=100)
print(q1.to_cl())
print(q2.to_cl())
print(res.get_counts())

[0]
[0]
[{"00": 47}, {"11": 53}]


自定义量子门

In [13]:
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, CNOT, gate, All, Measure

@gate
def my_gate(q):
    H * q[0]
    CNOT * (q[0], q[1])

circuit = QCircuit()
q = circuit.allocate(2)
my_gate * q
All(Measure) * q
res = circuit.run(shots=100)
print(res.get_counts())


[{"00": 57}, {"11": 43}]
